<a href="https://colab.research.google.com/github/am6180/Computer-Vision/blob/master/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

from torchvision import transforms
import numpy as np
from helper import getBinaryfer13Data

import pandas as pd

In [36]:
X, Y = getBinaryfer13Data("fer3and4train.csv")
X = torch.tensor(X.reshape(12066,1,48,48), dtype=torch.float)
Y = torch.tensor(Y)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [0]:
dataX = torch.utils.data.DataLoader(X, 10)
dataY = torch.utils.data.DataLoader(Y, 10)

In [0]:
class CNN(nn.Module):
  def __init__(self):
      super(CNN, self).__init__()

      self.cnn_layers = nn.Sequential(
            # Defining a 2D convolution layer
            nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=1),
            nn.BatchNorm2d(6),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # Defining another 2D convolution layer
            nn.Conv2d(6, 12, kernel_size=5, stride=1, padding=1),
            nn.BatchNorm2d(12),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

      self.linear_layers = nn.Sequential(
            nn.Linear(1200, 2)
        )

    # Defining the forward pass    
  def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x


In [0]:
def correct_pred(preds, labels):
  return preds.argmax(dim=1).eq(labels).sum().item()

In [82]:
model = CNN()
Image_data = torch.utils.data.DataLoader(X, 16)
labels = torch.utils.data.DataLoader(Y, 16)
optimize = optim.Adam(model.parameters(), 0.01)
for i in range(20):
  total_loss = 0
  total_correct = 0
  Ittrer = iter(labels)
  for img in Image_data:
    label = next(Ittrer)
    # for label in labels:
    pred = model(img)
    loss = F.cross_entropy(pred, label.type(torch.LongTensor))

    optimize.zero_grad()
    loss.backward()
    optimize.step()

    total_loss += loss.item()
    total_correct += correct_pred(pred, label)
  print("epoch:", i, "total correct number:", total_correct)
print("Accuracy = ", Accuracy*100)


epoch: 0 total correct number: 8255
epoch: 1 total correct number: 9331
epoch: 2 total correct number: 9562
epoch: 3 total correct number: 9721
epoch: 4 total correct number: 9840
epoch: 5 total correct number: 9925
epoch: 6 total correct number: 10037
epoch: 7 total correct number: 10092
epoch: 8 total correct number: 10143
epoch: 9 total correct number: 10193
epoch: 10 total correct number: 10275
epoch: 11 total correct number: 10263
epoch: 12 total correct number: 10348
epoch: 13 total correct number: 10407
epoch: 14 total correct number: 10402
epoch: 15 total correct number: 10398
epoch: 16 total correct number: 10472
epoch: 17 total correct number: 10482
epoch: 18 total correct number: 10561
epoch: 19 total correct number: 10569
Accuracy =  80.7


In [0]:
X1, Y1 = getBinaryfer13Data("fer3and4test.csv")
X1 = torch.tensor(X1.reshape(3000,1,48,48), dtype=torch.float)
Y1 = torch.tensor(Y1)

In [87]:
Image_data = torch.utils.data.DataLoader(X1, 100)
labels = torch.utils.data.DataLoader(Y1, 100)
for i in range(20):
  total_correct = 0
  Ittrer = iter(labels)
  for img in Image_data:
    label = next(Ittrer)
    pred = model(img)
    total_correct += correct_pred(pred, label)
  #print("epoch:", i, "total correct number:", total_correct)
Accuracy = total_correct/3000
print("Accuracy = ", Accuracy*100)

Accuracy =  79.36666666666666
